In [1]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [3]:
# inputs

# netdata demo space https://app.netdata.cloud/spaces/netdata-demo/rooms/apache/overview
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' # 
room_id = '9115ab73-317f-4a24-9233-f8d2993dbfc1'
context = 'apache.requests'

# last 10 minutes of data
after = -60*10
before = 0
freq = '10s' # 10 seconds

In [4]:
# get data from netdata cloud
df = get_data_cloud(space_id, room_id, context, after, before, freq=freq)
df = df.round(2) # round to 2 decimal places to save tokens
print(df.shape)
df.head()

(59, 1)


,requests
time,
2023-03-29 16:49:30,2595.51
2023-03-29 16:49:40,2028.89
2023-03-29 16:49:50,2.10
2023-03-29 16:50:00,2137.60
2023-03-29 16:50:10,747.05


In [5]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring an apache web server using Netdata Cloud.

Below is the some apache metrics for this server from {df.index.min()} to {df.index.max()}, aggregated to a {freq} frequency.

Does this server have a problem?

Metrics data:
{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring an apache web server using Netdata Cloud.

Below is the some apache metrics for this server from 2023-03-29 16:49:30 to 2023-03-29 16:59:10, aggregated to a 10s frequency.

Does this server have a problem?

Metrics data:
                     requests
time                         
2023-03-29 16:49:30   2595.51
2023-03-29 16:49:40   2028.89
2023-03-29 16:49:50      2.10
2023-03-29 16:50:00   2137.60
2023-03-29 16:50:10    747.05
2023-03-29 16:50:20   4223.25
2023-03-29 16:50:30      2.00
2023-03-29 16:50:40      2.00
2023-03-29 16:50:50   1988.01
2023-03-29 16:51:00   4967.10
2023-03-29 16:51:10      2.10
2023-03-29 16:51:20    344.81
2023-03-29 16:51:30   4343.34
2023-03-29 16:51:40    165.46
2023-03-29 16:51:50      2.10
2023-03-29 16:52:00   1616.83
2023-03-29 16:52:10   1473.57
2023-03-29 16:52:20      1.90
2023-03-29 16:52:30   2998.40
2023-03-29 16:52:40      1.90
2023-03-29 16:52:50      3.06
2023-03-29 16:53:00    415.

In [6]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [7]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [9]:
reply_content = completion.choices[0].message.content
print(reply_content)

It's hard to determine whether this server has a problem based on just the requests metric provided. However, we can make some observations:

- There seem to be some spikes in traffic at certain points in time (e.g. 16:49:30 and 16:51:00).
- There are also some periods of very low traffic (e.g. 16:49:50 and 16:50:30).
- The overall request rate seems to vary quite a bit, with highs of over 4,900 requests per 10 seconds and lows of just 1 or 2 requests.

To determine if the server has a problem, we would need to look at other metrics such as response times, error rates, CPU/memory usage, etc. It's also possible that this is normal behavior for the server depending on its usage patterns.
